In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Assuming you have already mounted your Google Drive as shown in your provided code
df = pd.read_excel('/content/drive/MyDrive/BI/processed_ticket.xlsx')

# Now you can work with the DataFrame 'df'
# Example: Print the first 5 rows
print(df.head())

            orderid cashier            saledate  total  customerid  \
0  10006052019B0225  emp002 2019-05-06 16:40:43  90000  0000000014   
1  10006052019B0225  emp002 2019-05-06 16:40:43  90000  0000000014   
2  10022052019B0167  emp011 2019-05-22 20:30:28  90000  0000000034   
3  10022052019B0167  emp011 2019-05-22 20:30:28  90000  0000000034   
4  10130052019B0141  emp005 2019-05-30 19:28:44  90000  0000000034   

   ticketcode       date      time slot  room  \
0    20074925 2019-05-06  18:15:00  G04     3   
1    20074924 2019-05-06  18:15:00  G03     3   
2    20097042 2019-05-22  20:45:00  E04     2   
3    20097041 2019-05-22  20:45:00  E03     2   
4    20106761 2019-05-30  19:30:00  F04     1   

                                      film slot type ticket type  \
0                        AVENGERS: HỒI KẾT       ĐƠN  Thành viên   
1                        AVENGERS: HỒI KẾT       ĐƠN  Thành viên   
2  JOHN WICK 3: CHUẨN BỊ CHIẾN TRANH (C18)       ĐƠN  Thành viên   
3  JOHN WICK

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35474 entries, 0 to 35473
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   orderid       35474 non-null  object        
 1   cashier       35474 non-null  object        
 2   saledate      35378 non-null  datetime64[ns]
 3   total         35474 non-null  int64         
 4   customerid    35474 non-null  object        
 5   ticketcode    35474 non-null  int64         
 6   date          35474 non-null  datetime64[ns]
 7   time          35474 non-null  object        
 8   slot          35474 non-null  object        
 9   room          35474 non-null  int64         
 10  film          35474 non-null  object        
 11  slot type     35474 non-null  object        
 12  ticket type   35474 non-null  object        
 13  ticket price  35474 non-null  int64         
 14  popcorn       35378 non-null  object        
dtypes: datetime64[ns](2), int64(4), obje

In [ ]:
df = df[df['orderid'].notnull()]

In [ ]:
import pandas as pd
import numpy as np
import random
import string

# Giả sử df đã được đọc từ file
# df = pd.read_csv("data.csv")

# Chỉ giữ lại các cột cần thiết
df_selected = df[['orderid', 'cashier', 'saledate', 'customerid', 'date', 'time', 'room', 'film', 'ticketcode', 'slot', 'ticket price', 'total', 'popcorn','slot type','ticket type']]

# Tính tổng giá vé theo orderid
total_ticket_price = df_selected.groupby('orderid')['ticket price'].sum().reset_index()
total_ticket_price.rename(columns={'ticket price': 'total_ticket_price'}, inplace=True)

# Merge tổng giá vé vào df_selected
df_selected = df_selected.merge(total_ticket_price, on='orderid', how='left')

# Tính chênh lệch
df_selected['difference'] = df_selected['total'] - df_selected['total_ticket_price']

df_selected['discount'] = 0  # Khởi tạo cột discount
df_selected['have_discount'] = "Không"  # Mặc định là "Không"

# Xác định các mức cần thêm dòng mới
adjustment_values = [45000, 90000, 75000, 160000]

def generate_unique_slot(existing_slots):
    while True:
        row = random.choice(string.ascii_uppercase[:8])  # Chọn ngẫu nhiên từ A-H
        number = random.randint(1, 17)  # Chọn số từ 1 đến 17
        new_slot = f"{row}{number}"
        if new_slot not in existing_slots:
            return new_slot

# Danh sách chứa dữ liệu cần thêm
new_rows = []

for _, row in df_selected.iterrows():
    if row['difference'] in adjustment_values:
        existing_slots = df_selected[(df_selected['date'] == row['date']) &
                                     (df_selected['time'] == row['time']) &
                                     (df_selected['room'] == row['room']) &
                                     (df_selected['film'] == row['film'])]['slot'].tolist()

        new_rows.append({
            'orderid': row['orderid'],
            'cashier': row['cashier'],
            'saledate': row['saledate'],
            'customerid': row['customerid'],
            'date': row['date'],
            'time': row['time'],
            'room': row['room'],
            'film': row['film'],
            'ticket type': row['ticket type'],
            'slot type': row['slot type'],
            'ticketcode': np.random.randint(10000, 99999),
            'slot': generate_unique_slot(existing_slots),
            'ticket price': row['difference'],
            'total': row['total'],
            'popcorn': row['popcorn'],
            'total_ticket_price': row['total_ticket_price'],
            'difference': row['difference'],
            'discount': 0,
            'have_discount': "Không"
        })
    elif row['difference'] == 225000:
        additional_tickets = []
        remaining_discount = 225000
        while remaining_discount > 0:
            ticket_price = 75000 if remaining_discount >= 75000 else 45000
            remaining_discount -= ticket_price
            existing_slots = df_selected[(df_selected['date'] == row['date']) &
                                         (df_selected['time'] == row['time']) &
                                         (df_selected['room'] == row['room']) &
                                         (df_selected['film'] == row['film'])]['slot'].tolist()
            additional_tickets.append({
                'orderid': row['orderid'],
                'cashier': row['cashier'],
                'saledate': row['saledate'],
                'customerid': row['customerid'],
                'date': row['date'],
                'time': row['time'],
                'room': row['room'],
                'film': row['film'],
                'ticketcode': np.random.randint(10000, 99999),
                'slot': generate_unique_slot(existing_slots),
                'ticket price': ticket_price,
                'total': row['total'],
                'popcorn': row['popcorn'],
                'total_ticket_price': row['total_ticket_price'],
                'difference': row['difference'],
                'discount': 0,
                'have_discount': "Không"
            })
        new_rows.extend(additional_tickets)
    elif row['difference'] != 0:
        df_selected.loc[df_selected['orderid'] == row['orderid'], 'discount'] = row['difference']
        df_selected.loc[df_selected['orderid'] == row['orderid'], 'have_discount'] = "Có"

# Tạo DataFrame từ danh sách mới và thêm vào df_selected
if new_rows:
    df_new = pd.DataFrame(new_rows)
    df_selected = pd.concat([df_selected, df_new], ignore_index=True)

# Xuất kết quả
# df_selected.to_csv("output.csv", index=False)
print(df_selected.head())


            orderid cashier            saledate  customerid       date  \
0  10006052019B0225  emp002 2019-05-06 16:40:43  0000000014 2019-05-06   
1  10006052019B0225  emp002 2019-05-06 16:40:43  0000000014 2019-05-06   
2  10022052019B0167  emp011 2019-05-22 20:30:28  0000000034 2019-05-22   
3  10022052019B0167  emp011 2019-05-22 20:30:28  0000000034 2019-05-22   
4  10130052019B0141  emp005 2019-05-30 19:28:44  0000000034 2019-05-30   

       time  room                                     film  ticketcode slot  \
0  18:15:00     3                        AVENGERS: HỒI KẾT    20074925  G04   
1  18:15:00     3                        AVENGERS: HỒI KẾT    20074924  G03   
2  20:45:00     2  JOHN WICK 3: CHUẨN BỊ CHIẾN TRANH (C18)    20097042  E04   
3  20:45:00     2  JOHN WICK 3: CHUẨN BỊ CHIẾN TRANH (C18)    20097041  E03   
4  19:30:00     1                   NGÔI ĐỀN KỲ QUÁI (C18)    20106761  F04   

   ticket price  total popcorn slot type ticket type  total_ticket_price  \
0   

In [ ]:
# Loại bỏ dấu chấm ở cuối các giá trị trong cột customerid
df_selected['customerid'] = df_selected['customerid'].astype(str).str.rstrip('.')

# Remove columns 'difference' and 'total_ticket_price'
df_selected = df_selected.drop(['difference', 'total_ticket_price'], axis=1)

# Xuất file vào /content/drive/MyDrive/BI/
df_selected.to_excel('/content/drive/MyDrive/BI/processed_ticket_cleaned.xlsx', index=False)